In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt 
import pickle 
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind,  shapiro, mannwhitneyu, pearsonr
import seaborn as sb
import matplotlib.colors
from tqdm import tqdm 

N_episode = 300
FIGSIZE = (16,9)
colors10 = ['darkred', 'crimson', 'salmon', 'gold', 'lime', 
          'darkgreen', 'navy', 'royalblue', 'darkviolet', 'magenta'
         ]
colors = ['crimson','gold','darkgreen','royalblue', 'darkviolet']

LW = 5

In [2]:
def plot_performance(data, X=None, names=None, title="", ylabel="performance", 
                     minimum = -np.inf, maximum=np.inf, log=False, double=False, xlabel="learning episodes"):
    """
    data = [n_param, n_run, episode_length]
    """
    
    if names is None:
        names = [str(j) for j in range(len(data))]
    compute_X = (X is None)
    if len(data)<6:
        ncol = [1,1,2,2,3][len(data)-1] 
    else:
        ncol = len(data)//2
    fig, ax = plt.subplots(figsize=FIGSIZE)
    handles = []
    for i in range(len(data)):
        if (double and i>=len(data)/2):
            linestyle = ":"
            color = colors[i%(len(data)//2)]
        else:
            linestyle = "-"
            color = colors[i%len(colors)]
        
        mean, std = np.mean(data[i], axis=0), np.std(data[i], axis=0)
        mean_p_std = np.minimum(mean+std, maximum)
        mean_m_std = np.maximum(mean-std, minimum)
        
        if compute_X:
            X = [x for x in range(len(mean))]

        for run in range(len(data[i])):
            plt.plot(X, data[i][run], color=color, lw=2, alpha=0.3, linestyle=linestyle)
        plt.plot(X,mean, color = color,lw=LW, linestyle=linestyle)
       # ax.fill_between(X,mean_p_std , mean_m_std, color= color, alpha=0.15)
       # plt.plot(X, mean_p_std, color=color, alpha=0.2)
        #plt.plot(X, mean_m_std, color=color, alpha=0.2)
        handles.append( mlines.Line2D([], [], color = color, linestyle=linestyle, label= names[i]))
    if log:
        plt.yscale("log")
        #if (X[1]-X[0])!=(X[-1]-X[-2]):
        #    plt.xscale("log")
    else:
        if minimum != -np.inf and maximum!=np.inf:
            plt.ylim(minimum,maximum)
        elif minimum != -np.inf:
            plt.ylim(minimum)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(handles=handles, numpoints = 2,ncol=1,bbox_to_anchor=(1.01, 1.))
    plt.show()

In [3]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

def compute_test(data, coef=1, N=5):
    pairs = []
    for i in range(N-1):
        for j in range(i+1,N):
            pairs.append((i,j))
    res = [[] for _ in range(N)]
    cells = [[ '' for _ in range(N-1)] for _ in range(N-1)]
    for (param1,param2) in pairs:
        data1 = np.array(data[param1])
        data2 = np.array(data[param2])
        stat, p_t = ttest_ind(data1,data2)
        p_t = p_t*coef
        mean1 = np.median(data1)
        mean2 = np.median(data2)
        d = truncate(mean1/mean2, 2)
        if d=='0.00' or d=='-0.00':
            d = truncate( mean1/mean2, 3)
            if d=='0.000' or d=='-0.000':
                d = truncate( mean1/mean2, 4)
                
        if p_t < 0.001:
            res.append((param1, param2, '***'))
            cells[param1][N-1-param2] = d+'\n***'
        elif p_t < 0.01:
            res.append((param1, param2, '**'))
            cells[param1][N-1-param2] = d+'\n**'
        elif p_t < 0.05:
            res.append((param1, param2, '*'))
            cells[param1][N-1-param2] = d+'\n*'
        else:
            cells[param1][N-1-param2] = "ns"
    return res,cells

def plot_boxplot(data, names, ylabel="performance", ylim=None, title="", log=False):
    N = len(data)
    stat,cells = compute_test(data, 3*N*(N-1)/2, N)
    
    plt.subplots(figsize=(12,10))
    
    bplot = sb.boxplot(data=data)
    for i in range(N):
        bplot.artists[i].set_facecolor(colors[i])
    if np.size(data) < 1000:
        sb.swarmplot( data=data, color='black', edgecolor='black',size=7)
    
    rows = names[:N-1]
    columns = [names[i] for i in range(N-1,0,-1)]
    cell_text = cells
    cellColours = [['white' if N-1-i>j else 'lightgrey' for j in range(N-1)] for i in range(N-1) ]
    the_table = plt.table(cellText=cell_text,
                          rowLabels=rows,
                          cellColours= cellColours,
                          rowColours=colors[:N-1],
                          colColours=[ colors[i] for i in range(N-1,0,-1)],
                          colLabels=columns,
                          cellLoc = 'center',
                          bbox=(1.13,0.1,0.5,0.9))
    
    plt.xticks(range(N), names)
    if log:
        plt.yscale('log')
    if not ylim is None:
        plt.ylim(ylim)
    plt.ylabel(ylabel)
    plt.title(title)

def plot_model_training(subdirs, names):
    Train, Test = [], []
    for param in subdirs:
        trains, tests = [], []
        min_length = np.inf 
        for folder in param:
            data = scipy.io.loadmat(folder)
            test = data['test_error']
            train = data['train_error']
            trains.append(np.mean(train,axis=1))
            tests.append(np.mean(test,axis=1))
            if min_length > len(test):
                min_length = len(test)
        tests = [ test[:min_length] for test in tests]
        trains = [ train[:min_length] for train in trains]
        Test.append(np.copy(tests))
        Train.append(np.copy(trains))
    Data = Train + Test
    Names = ["Training error "+x for x in names] + ["Testing error "+x for x in names]
    plot_performance(Data, names=Names, double=True, log=True, ylabel="Error", title="One Step Prediction Error")

In [4]:
def add_samples(samples, type="D", H=20):
    inputs, outputs = [], []
    if type == "D":
        acs = samples['acs']
        obs = samples['obs']
        indexes = []
        for i in range(len(acs)):
            T = len(obs[i])
            for t in range(len(acs[i])):
                if t < T-H:
                    my_index = len(inputs)
                    indexes.append(my_index)
                inputs.append(np.concatenate((obs[i][t][:28], obs[i][t][30:31], acs[i][t])))
                outputs.append(obs[i][t + 1]- obs[i][t])
        indexes = np.array(indexes)
    else:
        indexes = None
        ctrl = samples['controller']
        obs = samples['obs']
        for i in range(len(obs)):
            T = len(obs[i])
            for t in range(0, T-H):
                inputs.append(np.concatenate((obs[i][t][:28], obs[i][t][30:31], ctrl[i], [(t%25)/25])))
                outputs.append(obs[i][t + H][28:31]- obs[i][t][28:31])     
    return np.array(inputs), np.array(outputs), indexes

In [7]:
path = "exp/log/Saved/ControlledMinitaur/"
file = "/logs.mat"

test1 = "2019-11-22--09:17:2155470"

folder = test1

In [8]:
with open(path+folder+file, 'rb') as f:
    data = pickle.load(f)

ValueError: invalid literal for int() with base 10: 'AB 5.0 MAT-file Platform: posix, Created on: Fri Nov 22 09:29:37 2019'